# Reto 1

In [1]:
#Importar librerías
import math
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Se leen los datos utilizando readcsv de Pandas

In [2]:
#Leer el archivo con los datos
data_pulsar = pd.read_csv('./data/HTRU_2.csv', header=None)

Se separan los datos en "x" y "y". Se normaliza x

In [3]:
#La variable dependiente es la última columna, las independientes son las anteriores.
#Se normaliza las independientes y se convierte nuevamente en un DataFrame de Pandas
x= pd.DataFrame.from_records(preprocessing.normalize(data_pulsar.iloc[:,0:8]))
y= data_pulsar.iloc[:,8:9]

Se parten los datos en entrenamiento y test

In [4]:
#Se divide el archivo para entrenamiento y test. Se reserven 10000 datos para test
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 10000, random_state = 0)

In [5]:
#Se concatenan los datos de test
newData= pd.concat([xTrain,yTrain], axis = 1)

Se escogen aleatoriamente diferentes tamaños de datos para entrenar

In [6]:
#De estos datos concatenados se escogen aleatoriamente 100,500,1000 y 5000 para diferentes modelos. 
#De aquí se vuelven a separar en x y y
dataTrain1= newData.sample(100, random_state = 0)
xTrain1= dataTrain1.iloc[:,0:8]
yTrain1= dataTrain1.iloc[:,8]
dataTrain2= newData.sample(500, random_state = 0)
xTrain2= dataTrain2.iloc[:,0:8]
yTrain2= dataTrain2.iloc[:,8]
dataTrain3= newData.sample(1000, random_state = 0)
xTrain3= dataTrain3.iloc[:,0:8]
yTrain3= dataTrain3.iloc[:,8]
dataTrain4= newData.sample(5000, random_state = 0)
xTrain4= dataTrain4.iloc[:,0:8]
yTrain4= dataTrain4.iloc[:,8]

Se entrenan los modelos

In [7]:
#Se entrenan los modelos respectivos utilizando la librería de regresión logística de skelearn
logisticRegr1 = LogisticRegression(solver='liblinear')
logisticRegr1.fit(xTrain1, yTrain1)
print(logisticRegr1.coef_)
logisticRegr2 = LogisticRegression(solver='liblinear')
logisticRegr2.fit(xTrain2, yTrain2)
print(logisticRegr2.coef_)
logisticRegr3 = LogisticRegression(solver='liblinear')
logisticRegr3.fit(xTrain3, yTrain3)
print(logisticRegr3.coef_)
logisticRegr4 = LogisticRegression(solver='liblinear')
logisticRegr4.fit(xTrain4, yTrain4)
print(logisticRegr4.coef_)

[[-1.3471877  -0.12043274  0.10001708  0.50525963  0.79415348  0.90285417
  -0.08293925 -1.62638016]]
[[-3.08949271  0.5681149   0.44026083  2.24643966  1.24674773  2.99543923
   0.01975481 -2.03796584]]
[[-3.64709138  2.04694934  0.70306718  3.01305858  0.32916592  4.24427233
   0.30359388 -1.77718205]]
[[-5.44700152  4.90003648  1.77036411  6.86075577 -1.17970812  7.49200128
   1.36362609 -1.23668302]]


Se construye una función de éxito para evaluar el modelo.
Esta función es la que será utilizada para evaluar el modelo con descenso de gradiente también y así poder comprar

In [8]:
#Definición de función sigmoide para usarse posteriormente
def sigmoid(x):
     return 1 / (1 + math.exp(-x))

In [9]:
#Función que calcula el error final de clasificación. 
#Como tal lo que devuelve es el porcentaje de datos exitosamente clasificados
def calc_exito(nX,nY,w):
    a=nX.to_numpy()
    y=nY.to_numpy()
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    err=0
    for i in range(0, numData):
        prob1=sigmoid((x[i]*w).sum())
        #El porcentaje de error aumenta si se clasificó mal tanto como 1 cómo como 0
        if prob1>0.5 and y[i]==0:
            err=err+1
        #Se asigna el 0.5 como valor inclusivo hacia el 0 dado que el porcentaje de 0's es mucho mayor.
        if prob1<=0.5 and y[i]==1:
            err=err+1
    return 1-(err/numData)
    

Se evalúan los datos con la función score de skelearn

In [10]:
#Resultado de test de modelo 1
score1 = logisticRegr1.score(xTest, yTest)
print(score1)

0.9144


In [11]:
#Resultado de test de modelo 2
score2 = logisticRegr2.score(xTest, yTest)
print(score2)

0.9576


In [12]:
#Resultado de test de modelo 3
score3 = logisticRegr3.score(xTest, yTest)
print(score3)

0.9618


In [13]:
#Resultado de test de modelo 4
score4 = logisticRegr4.score(xTest, yTest)
print(score4)

0.9695


Se evalúa el modelo con la función personalizada de éxito

In [14]:
ws=logisticRegr1.coef_[0]

score1 = calc_exito(xTest, yTest,np.append(ws,logisticRegr1.intercept_))
print(score1)

0.9144


In [15]:
ws=logisticRegr2.coef_[0]

score2 = calc_exito(xTest, yTest,np.append(ws,logisticRegr2.intercept_))
print(score2)

0.9576


In [16]:
ws=logisticRegr3.coef_[0]

score3 = calc_exito(xTest, yTest,np.append(ws,logisticRegr3.intercept_))
print(score3)

0.9618


In [17]:
ws=logisticRegr4.coef_[0]

score4 = calc_exito(xTest, yTest,np.append(ws,logisticRegr4.intercept_))
print(score4)

0.9695


## Resultados
Como se puede observar, todos los modelos tuvieron menos de 10% de error, lo que implicaría que los datos se comportan de manera altamente lineal. A medida que aumenta el número de dato se observa que aumenta la tasa de éxito, lo cual concuerda con la teoría. Además, se observa que ambas funciones de error utilizadas dan como resultado el mismo valor, por lo que esta función personalizada puede ser usada para evaluar el modelo hecho a mano en el reto 3.
Sin embargo, se debe tener en cuenta que la mayoría de valores yi de los datos son 0. Por lo cual un modelo que tiende a clasificar los datos como 0 posiblemente dé mejores resultados. Aunque se debe tener en cuenta que la tasa de éxito fue de más de 90% con suficientes datos, que es lo que pasaría si realmente los datos de entrenamiento y validación se escogen de manera aleatoria y el modelo solamente indicara 0 para cualquier dato.